In [ ]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
from IPython.display import clear_output

srchtext = input("검색어 : ")
srchtext = urllib.parse.quote(srchtext)
result = []
for pagenum in range(1,100):
  url = f'http://www.yes24.com/Product/Search?domain=ALL&query={srchtext}&page={pagenum}'
  clear_output(wait= True)
  print(f'크롤링 페이지 : {url}', end = '\n')

  html = urllib.request.urlopen(url)
  soup = BeautifulSoup(html,'html.parser')
  litag_list = soup.select('#yesSchList > li')
  if len(litag_list) == 0: break
  
  for temp in litag_list:  
    try:
      auth =  temp.select('div > div.item_info > div.info_row.info_pubGrp > span.authPub.info_auth > a')
      auth = ','.join([au.string for au in auth])

      title = temp.select('div > div.item_info > div.info_row.info_name > a.gd_name')[0].string  
      public = temp.select('div > div.item_info > div.info_row.info_pubGrp > span.authPub.info_pub > a')[0].string  
      date = temp.select('div > div.item_info > div.info_row.info_pubGrp > span.authPub.info_date')[0].string  
      result.append([title,auth,public,date])  
    except Exception as e:
      print(e)      

# 출력
df = pd.DataFrame(result,columns=['책 제목','저자','출판사','출판일'])    
df_sorted = df.sort_values(by = '출판일',ascending=False)
df_sorted.reset_index(drop=True,inplace=True)
df_sorted.to_csv(f'./yes24_{srchtext}.csv')
df_sorted.head()